In [14]:
from PIL import Image

from keras.models import load_model
import numpy as np
from numpy import asarray
from numpy import expand_dims
import requests
from io import BytesIO
import pickle
import cv2
from keras_facenet import FaceNet

In [15]:
#HaarCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
HaarCascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))
#MyFaceNet = load_model('facenet_keras.h5')
MyFaceNet = FaceNet()

In [16]:
myfile = open("data.pkl", "rb")
database = pickle.load(myfile)
myfile.close()
url ="http://192.168.1.16/cam-lo.jpg"

In [ ]:
while True:
    # Lấy ảnh từ ESP32-CAM
    response = requests.get(url)
    img_data = response.content
    img_array = np.array(bytearray(img_data), dtype=np.uint8)
    gbr1 = cv2.imdecode(img_array, -1)

    # Kiểm tra xem ảnh có bị lỗi không
    if gbr1 is None:
        print("Không thể lấy ảnh từ ESP32-CAM!")
        break

    # Phát hiện khuôn mặt trong ảnh
    wajah = HaarCascade.detectMultiScale(gbr1, 1.1, 4)

    if len(wajah) > 0:
        # Nếu phát hiện khuôn mặt
        x1, y1, width, height = wajah[0]
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height

        # Chuyển ảnh từ OpenCV (BGR) sang RGB
        gbr = cv2.cvtColor(gbr1, cv2.COLOR_BGR2RGB)
        gbr = Image.fromarray(gbr)  # Chuyển từ OpenCV sang PIL
        gbr_array = asarray(gbr)

        # Cắt khuôn mặt trong ảnh
        face = gbr_array[y1:y2, x1:x2]
        face = Image.fromarray(face)
        face = face.resize((160, 160))  # Đảm bảo kích thước face chuẩn
        face = asarray(face)
        face = expand_dims(face, axis=0)
        
        # Lấy đặc trưng khuôn mặt từ mô hình (ví dụ: FaceNet)
        # signature = MyFaceNet.predict(face)
        signature = MyFaceNet.embeddings(face)

        # Tìm người có đặc trưng gần nhất trong cơ sở dữ liệu
        min_dist = 10
        identity = 'Unknown'  # Mặc định là không nhận diện được
        for key, value in database.items():
            dist = np.linalg.norm(value - signature)
            if dist < min_dist:
                min_dist = dist
                identity = key

        # Hiển thị kết quả
        if identity == 'Unknown':
            # Không nhận diện được -> chữ màu đỏ
            cv2.putText(gbr1, "Unknown", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        else:
            # Nhận diện thành công -> chữ màu xanh
            cv2.putText(gbr1, identity, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
        # Vẽ khung xanh quanh khuôn mặt
        cv2.rectangle(gbr1, (x1, y1), (x2, y2), (0, 255, 0), 2)
    else:
        # Nếu không phát hiện được khuôn mặt -> Hiển thị thông báo
        cv2.putText(gbr1, "No face detected", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    # Hiển thị kết quả
    cv2.imshow('res', gbr1)

    # Nhấn ESC để thoát
    k = cv2.waitKey(5) & 0xFF
    if k == 27:  # ESC
        break

cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 